# Download slides from Tau to Nissl registration

In [1]:
import os
import subprocess
import sys

import json
import pandas as pd
from tqdm import tqdm

# https://github.com/pyushkevich/histoannot.git
# git clone the GitHub repository and add the path to the sys.path
# This makes sure you are using the latest version of the code
sys.path.append('/Users/cathalye/Packages/histoannot/')
import phas.client.api as phas

## Connect to server
User input needed - `PHAS_URL`, `PRIVATE_KEY`

In [2]:
# To get the API key you need to go to https://histo.itksnap.org/auth/api/generate_key
PHAS_URL='https://histo.itksnap.org'
PRIVATE_KEY = '/Users/cathalye/.private/histoitk_api_key.json'

conn = phas.Client(PHAS_URL, PRIVATE_KEY)
proj_listing = pd.DataFrame(conn.project_listing())
proj_listing

,id,admin,disp_name,desc,nslides,nblocks,nspecimens
0,mtl_bf,0,MTL Blockface,Blockface images from the MTL project,28934,81,21
1,mtl,0,MTL Serial Histology,Serially sectioned medial temporal lobe specim...,19248,376,93


In [3]:
task_listing = pd.DataFrame(conn.task_listing('mtl'))
task_listing

,id,name,desc,mode,nspecimens,nblocks,nslides
0,1,Tanglethon 2019,Training a deep learning classifier to recogni...,dltrain,92,356,5214
1,2,Anatomical Labeling - UCLM,Labeling anatomical boundaries and regions by ...,annot,93,376,19248
2,7,Browse,Browse the slide collection,browse,93,376,19248
3,23,Diagnostics,Semiquantiatative rating of selected diagnosti...,dltrain,85,177,876
4,29,Tardython 2022 (TDP-43),Training a deep learning classifier to recogni...,dltrain,75,154,260
5,37,Stereology: Nissl [Deprecated],Stereology for validation of quantitative Niss...,dltrain,89,351,12462
6,38,NISSL stereology sampling ROIs,ROIs for stereology validation,sampling,93,376,19248
7,39,Diagnostic slide sampling,Diagnostic slide sampling,sampling,85,177,876


### Choose task ID
User input needed - `TASK_ID` (choose the id number from `task_listing` dataframe above)

In [4]:
TASK_ID = 39

# Create a task object to pass to Slide class for downloading thumbnails
task = phas.Task(conn, TASK_ID)


In [5]:
manifest = pd.DataFrame(task.slide_manifest())
manifest.head(5)

,id,block_id,section,slide,stain,slide_name,slide_ext,project,specimen,block_name,specimen_private,specimen_public,n_sampling_rois
0,376,12,14,6,Tau,285310,svs,mtl,275,HR2a,HNL-11-15,HNL-11-15R,0
1,422,12,14,10,NISSL,285538,svs,mtl,275,HR2a,HNL-11-15,HNL-11-15R,0
2,342,10,4,6,Tau,285035,svs,mtl,275,HR4a,HNL-11-15,HNL-11-15R,0
3,2179,10,4,10,NISSL,HNL-11-15_HR4a_NISSL_4,tif,mtl,275,HR4a,HNL-11-15,HNL-11-15R,0
4,38964,57,29,6,SGR,HNL-28-17_HL2a_SGR_29_diag,tif,mtl,276,HL2a,HNL-28-17,HNL-28-17L,0


In [6]:
# Check for Nissl slides with ROIs
manifest_nissl_rois = manifest[(manifest['n_sampling_rois'] != 0) & (manifest['stain']=='NISSL')]
manifest_nissl_rois.head(5)


,id,block_id,section,slide,stain,slide_name,slide_ext,project,specimen,block_name,specimen_private,specimen_public,n_sampling_rois
54,1726,41,20,10,NISSL,HNL-34-18_HR3a_NISSL_20,tif,mtl,280,HR3a,HNL-34-18,HNL-34-18R,1
282,47619,6160,12,10,NISSL,UP-41_HR2a_NISSL_12,tif,mtl,3864,HR2a,INDD102115,UP41-22R,4
289,47662,6205,10,10,NISSL,UP-41_HR3a_NISSL_10,tif,mtl,3864,HR3a,INDD102115,UP41-22R,8
296,46657,5190,20,10,NISSL,UP-42_HL2a_NISSL_20,tif,mtl,2895,HL2a,INDD102328,UP42-22L,3
303,46701,5230,24,10,NISSL,UP-42_HL3a_NISSL_24,tif,mtl,2895,HL3a,INDD102328,UP42-22L,9


## Download the slides and save the files to specimen folder

User input needed - `ROOT_DIR`

In [ ]:
ROOT_DIR = (
    "/Users/cathalye/Library/CloudStorage/Box-Box/chinmayee/202411_pmam_late/data"
)

with tqdm(total=manifest_nissl_rois.shape[0]) as pbar:
    for index, row in manifest_nissl_rois.iterrows():
        pbar.update(1)
        # Get the specimen, block, section and slide id for the Nissl slide
        specimen = row["specimen_private"]
        os.makedirs(f"{ROOT_DIR}/{specimen}", exist_ok=True)
        block = row["block_name"]
        os.makedirs(f"{ROOT_DIR}/{specimen}/{block}", exist_ok=True)
        section = row["section"]
        nissl_slide_id = row["id"]
        try:
            # Check if there is a tau slide for the same specimen, block and section
            tau_slide_id = manifest[
                (manifest["specimen_private"] == specimen)
                & (manifest["block_name"] == block)
                & (manifest["section"] == section)
                & (manifest["stain"] == "Tau")
            ]["id"].values[0]
        except IndexError:
            print(
                f"Tau slide not found for Specimen: {specimen}, Block: {block}, Section: {section}"
            )
            tau_slide_id = None

        # Download Nissl slide
        nissl_slide = phas.Slide(task, nissl_slide_id)
        nissl_fname = f"{ROOT_DIR}/{specimen}/{block}/nissl_slide_thumbnail.nii.gz"
        if not os.path.exists(nissl_fname):
            try:
                nissl_slide.thumbnail_nifti_image(nissl_fname)
            except Exception as e:
                print(f"Error downloading Nissl slide {nissl_slide_id}")

        # Download Tau slide
        if tau_slide_id:
            tau_slide = phas.Slide(task, tau_slide_id)
            tau_fname = f"{ROOT_DIR}/{specimen}/{block}/tau_slide_thumbnail.nii.gz"
            if not os.path.exists(tau_fname):
                try:
                    tau_slide.thumbnail_nifti_image(tau_fname)
                except Exception as e:
                    print(f"Error downloading Tau slide {tau_slide_id}")
        else:
            tau_slide_id = 0

        # Download ROI for Nissl slide
        ROI_task = phas.SamplingROITask(conn, TASK_ID)
        ROI_fname = f"{ROOT_DIR}/{specimen}/{block}/nissl_sampling_roi.nii.gz"
        if not os.path.exists(ROI_fname):
            try:
                ROI_task.slide_sampling_roi_nifti_image(nissl_slide_id, ROI_fname)
            except Exception as e:
                print(f"Error downloading ROI for Nissl slide {nissl_slide_id}")

        block_dict = {
            "specimen": specimen,
            "block": block,
            "section": section,
            "nissl_slide_id": int(nissl_slide_id),
            "tau_slide_id": int(tau_slide_id),
        }
        with open(
            f"{ROOT_DIR}/{specimen}/{block}/block_info.json", "w", encoding="utf-8"
        ) as f:
            json.dump(block_dict, f, ensure_ascii=False, indent=4)

 17%|█▋        | 8/46 [00:03<00:18,  2.09it/s]

Tau slide not found for Specimen: INDD102374, Block: HL1a, Section: 45


 24%|██▍       | 11/46 [00:04<00:15,  2.19it/s]

Error downloading Nissl slide 37210


 26%|██▌       | 12/46 [00:05<00:19,  1.74it/s]

Error downloading ROI for Nissl slide 37210


 50%|█████     | 23/46 [00:10<00:10,  2.11it/s]

Tau slide not found for Specimen: INDD117243, Block: HR3a, Section: 18


100%|██████████| 46/46 [00:21<00:00,  2.09it/s]
